In [1]:
import requests
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
#API KEY LAST FM

API_KEY = '7297aea95a89b3bf151286c42842cdf2'
BASE_URL = 'http://ws.audioscrobbler.com/2.0/'

In [3]:
# Cargar los datos desde un archivo existente (CSV o Excel)
# Reemplazar 'ruta_del_archivo.csv' o 'ruta_del_archivo.xlsx' con la ruta real del archivo
ruta_csv = 'total_artistas.csv'  # Archivo CSV con los 20000 artistas

# Leer el archivo CSV (descomentar la línea si se utiliza un archivo CSV)
df = pd.read_csv('./CSV/total_artistas.csv', header=None, names=['artista'])

num_filas = len(df)

df_lista_artistas = df.drop_duplicates(keep='first') # Eliminar duplicados

print(num_filas)
print(len(df_lista_artistas))
# Confirmar la carga de datos
print(df.head())

# Guardar nuevamente como un archivo CSV o Excel si es necesario
#df.to_csv('lista_datos.csv', index=False)

# Confirmar que los archivos han sido generados
print("Archivos 'lista_datos.csv' ha sido generado a partir del archivo de origen.")

8694
4960
         artista
0              0
1       DJ Snake
2  Maribou State
3         Xinobi
4           Rhye
Archivos 'lista_datos.csv' ha sido generado a partir del archivo de origen.


In [4]:
print(df.head())  # Primeras filas del DataFrame
print(df.shape)   # Tamaño del DataFrame (filas, columnas)


         artista
0              0
1       DJ Snake
2  Maribou State
3         Xinobi
4           Rhye
(8694, 1)


In [7]:
import time
resultados = [] # Lista para almacenar los resultados

for artista in df_lista_artistas['artista']:
    
    params_info = {
        "method": "artist.getInfo",
        "api_key": API_KEY,
        "format": "json",
        "artist": artista
    }
    try:
        response_info = requests.get(BASE_URL, params=params_info)
        response_info.raise_for_status()  # Verificar errores
        data_json = response_info.json()

        # Extraer solo los datos necesarios
        artist_info = data_json.get('artist', {})
        popularidad = artist_info.get('stats', {})  # Popularidad y estadísticas
        biografia = artist_info.get('bio', {}).get('summary')  # Biografía breve
        artistas_similares = [
            similar['name']
            for similar in artist_info.get('similar', {}).get('artist', [])
        ]
        # Agregar datos al listado de resultados
        resultados.append({
            "Artista": artista,
            "Reproducciones": popularidad.get('playcount'),
            "Oyentes": popularidad.get('listeners'),
            "Biografía": biografia,
            "Artistas Similares": ', '.join(artistas_similares)
        })

    except requests.exceptions.RequestException as e:
        print(f"Error al obtener información para el artista {artista}: {e}")

    time.sleep(0.5)  # Pausa de medio segundo entre solicitudes, sino nos tira errores de algunos artistas (la API tiene un límite)


Error al obtener información para el artista Ana Bacalhau: Expecting value: line 1 column 1 (char 0)


In [1]:
# Convertir resultados a un DataFrame
df_resultados = pd.DataFrame(resultados)

# Guardar en un archivo CSV
df_resultados.to_csv('./CSV/resultados_artistas3.csv', index=False, encoding='utf-8') #Tenemos que añadir lo del encoding porque al haber saltos de línea en la BIO y caracteres especiales, necesita soporte
df_resultados.to_excel('./EXCEL/resultados_artistas3.xlsx', index=False)

NameError: name 'pd' is not defined

In [22]:
##Listado de artistas de spotify
##Recorrer ese listado y hacer una peticion a last.fm con el nombre del artista
##Crear un listado para almacenar los diccionarios de los artistas
##Almacenar ese listado en un fichero